In [134]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [135]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
from sklearn.preprocessing import LabelEncoder


In [136]:

# Create synthetic dataset

data = pd.read_csv("fraudTest.csv", index_col=0)

data.head()


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [137]:
data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [138]:
data.dtypes


trans_date_trans_time     object
cc_num                     int64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
job                       object
dob                       object
trans_num                 object
unix_time                  int64
merch_lat                float64
merch_long               float64
is_fraud                   int64
dtype: object

In [139]:
# First, I'm converting 'trans_date_trans_time' and 'dob' into datetime type

data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
data['dob'] = pd.to_datetime(data['dob'])

In [140]:
# Now, about 'dob' (day of birth), we can get the age of the user
data['dob']= data['dob'].dt.year
data = data.rename(columns = {'dob': 'age'})


from datetime import datetime
data['age'] = datetime.now().year - data['age']

In [141]:

# Extract useful features from trans_date_trans_time
data['year'] = data['trans_date_trans_time'].dt.year
data['month'] = data['trans_date_trans_time'].dt.month
data['day'] = data['trans_date_trans_time'].dt.day
data['hour'] = data['trans_date_trans_time'].dt.hour
data['minute'] = data['trans_date_trans_time'].dt.minute


In [142]:

# Drop the original datetime column if not needed
data = data.drop(columns=['trans_date_trans_time'])


In [143]:

# Encode categorical features
label_encoder = LabelEncoder()
data['gender'] = label_encoder.fit_transform(data['gender'])
data['category'] = label_encoder.fit_transform(data['category'])


In [144]:
features.dtypes


year            int64
month           int64
day             int64
hour            int64
minute          int64
age             int64
amt           float64
gender          int64
category        int64
merch_lat     float64
merch_long    float64
dtype: object

In [145]:
# Identify features and target variable

features = data[["year","month", "day","hour","minute","age", "amt", "gender" ,"category", "merch_lat","merch_long"]]
target = data['is_fraud']  # The target variable


In [146]:

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardize the feature values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the XGBoost classifier
model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Train the model on the training data
model.fit(X_train_scaled, y_train)

# Save the trained model and scaler for real-time use
joblib.dump(model, 'fraud_detection_model.pkl')
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [147]:
pip install Flask


Note: you may need to restart the kernel to use updated packages.


In [148]:
import requests

url = 'http://127.0.0.1:8088/predict'
data = {
    'year': 2024,
    'month': 6,
    'day': 5,
    'hour': 8,
    'minute': 7,
    'age': 30,
    'amt': 15000.0,
    'gender': 1,
    'category': 6,
    'merch_lat': 34.052235,
    'merch_long': -118.243683
}

response = requests.post(url, json=data)
print(response.json())


{'is_fraud': 0}
